In [1]:
import undetected_chromedriver as uc
import os
import os
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from rapidfuzz import fuzz  # Make sure to install with: pip install rapidfuzz
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import shutil


SIMILARITY_THRESHOLD = 85 

def strip_non_bmp(text):
    return ''.join(c for c in text if ord(c) <= 0xFFFF)


In [2]:
profile_path = "/Users/ashhadulislam/automation-profile"
os.makedirs(profile_path, exist_ok=True)

options = uc.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_path}")
driver = uc.Chrome(options=options)
driver.get("https://www.youtube.com")

In [3]:

#time.sleep(10)  # wait for manual login if not using session

response_dir = "response_to_polarizing"
responded_dir='responded_to_polarizing'

for file in os.listdir(response_dir)[1:]:
    if not file.endswith(".json"):
        continue

    video_id = file.split('_')[0]
    video_url = f"https://www.youtube.com/watch?v={video_id}"

    with open(os.path.join(response_dir, file)) as f:
        data = json.load(f)

    polar_comment = data["polarizing_comment"]
    reply_text = data["balanced_response"]

    driver.get(video_url)
    time.sleep(10)

    # Scroll to load comments
    driver.execute_script("window.scrollTo(0, 800);")
    time.sleep(5)

    # Search for the polarizing comment
    comments = driver.find_elements(By.ID, 'content-text')
    target = None
    best_score = 0
    print(f'Looking for {polar_comment}')
    for c in comments:
        comment_text = c.text.strip()
        score = fuzz.ratio(polar_comment.strip(), comment_text)
        print(f'comment:{comment_text}, score: {score}')
        if score > SIMILARITY_THRESHOLD and score > best_score:
            target = c
            best_score = score

    
    if target:
        try:
            # Scroll to the target comment
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", target)
            time.sleep(2)
        
            # Get the containing comment block
            container = target.find_element(By.XPATH, "./ancestor::ytd-comment-thread-renderer")
        
            # Find the new reply button (based on your HTML)
            reply_button = container.find_element(By.XPATH, ".//ytd-button-renderer[@id='reply-button-end']//button")
            reply_button.click()
            time.sleep(2)
        
            # Wait and find the textbox
            textbox = driver.find_element(By.ID, "contenteditable-root")
            reply_text_clean = strip_non_bmp(reply_text)
            textbox.send_keys(reply_text_clean)
            time.sleep(1)
        
            # Click submit button
            submit = driver.find_element(By.ID, "submit-button")
            submit.click()
            print(f"✅ Replied to: {video_url}")
            # Move the file
            shutil.move(os.path.join(response_dir, file), os.path.join(responded_dir, file))
            print(f"📦 Moved {file} to {responded_dir}")

        
        except NoSuchElementException as e:
            print(f"⚠️ Couldn't find reply button for comment on {video_url}")
            print(e)
        
        except Exception as e:
            print(f"⚠️ Unexpected error for {video_url}")
            print(e)
            
    else:
        print(f"❌ Comment not found in: {video_url}")
    

Looking for You know the irony of Murray saying you're not an expert to talk about something? He talked about how bad Islam is, he doesn't any credential in Islamic studies yet he still does talk about it 😂
comment:You know the irony of Murray saying you're not an expert to talk about something? He talked about how bad Islam is, he doesn't any credential in Islamic studies yet he still does talk about it, score: 99.48186528497409
comment:Pro Israel rent boy Murray is an "expert", because the IDF took him on a tour of areas they selected. Makes perfect sense..., score: 42.138364779874216
comment:But Douglas is criticising islam like he's an alim and talking about the historical issues in the middle east like he has the credentials of Adnan Rashid on the topic., score: 43.767313019390585
comment:Dougless Murray is to journalism as Ben Shapiro is to academics, score: 31.906614785992215
comment:North Korea and Iran. You’re telling me he’s neeevveerr beeeeen thereeee??, score: 29.8507462686

In [ ]:
video_url

In [ ]:
target

In [ ]:
reply_button = target.find_element(By.XPATH, "../../../..//ytd-button-renderer[@id='reply-button']")